# Heatmaps

This notebook will make heatmaps for the X first or last fixations. You can change the amount and it will take that amount as the first or last fixations.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from math import sqrt, pi,exp, isnan
import math
import os 
root = '../../heatmaps/sections/'

#define the gaussian function for the heatmap (code from Pygaze)
def gaussian(w, h, s):
	W = np.zeros([h,w],dtype=float)
	dims = W.shape
	xo = w/2
	yo = h/2
	for i in range(w):
			for j in range(h):
				W[j,i] = np.exp(-1.0 * (((float(i)-xo)**2/(2*s*s)) + ((float(j)-yo)**2/(2*s*s)) ) )
	return(W)

#loop through the trust data and add gaussians (code from Pygaze)
#make sure to have columns Xloc and Yloc in your dataframe
def create_heatmap(fixations):
	#create empty heatmaps to add the fixation gaussians to
	heatmap = np.zeros(heatmapsize)
	for index, row in fixations.iterrows():
		x = int(strt + row['X'] - int(gwh/2))
		y = int(strt + row['Y'] - int(gwh/2))
		# correct Gaussian size if either coordinate falls outside of
		# display boundaries
		if (not gwh/2 < x < maxX-gwh/2-1) or (not gwh/2 < y < maxY-gwh/2-1):
			hadj=[0,gwh]
			vadj=[0,gwh]
			if 0 > x:
				hadj[0] = abs(x)
				x = 0
			elif maxX < x:
				hadj[1] = gwh - int(x-maxX)
			if 0 > y:
				vadj[0] = abs(y)
				y = 0
			elif maxY < y:
				vadj[1] = gwh - int(y-maxY)
			# add adjusted Gaussian to the current heatmap
			try:
				heatmap[y:y+vadj[1],x:x+hadj[1]] += gaus[vadj[0]:vadj[1],hadj[0]:hadj[1]] 
			except:
			# fixation was probably outside of display
				pass
		else:				
		# add Gaussian to the current heatmap
			heatmap[y:y+gwh,x:x+gwh] += gaus 

	# resize heatmap (Pygaze)
	heatmap = heatmap[strt:maxY+strt,strt:maxX+strt]

	# remove zeros
	lowbound = np.mean(heatmap[heatmap>0])
	heatmap[heatmap<lowbound] = np.nan
	return heatmap




In [2]:
def heatmap_section(exp, image, scene, amount=25):
    
    subfolder = f'{amount}_sections/scene{scene}_{image}/{exp}/'
    os.makedirs(root+subfolder, exist_ok=True)
    fix_control = fix[(fix['experience'] == exp)]
    print(len(fix_control))
    ams = math.floor(len(fix_control)/amount)
    print(ams)
    mod_len = len(fix_control)//ams
    print(mod_len)
    for i in range(mod_len):
        j = i
        fix_firstx = fix_control.apply(lambda x: x.iloc[j*ams: (j+1)*ams]).reset_index(drop=True)
        print(len(fix_control), j*ams,(j+1)*ams)
        heatmap = create_heatmap(fix_firstx)
    
    
        #plot the heatmap and save the image
        fig = plt.figure()
        imgplot = plt.imshow(background[startX:startX+dims[0], startY:startY+dims[1]])
        imgplot = plt.imshow(heatmap[startX:startX+dims[0], startY:startY+dims[1]], cmap='hot', alpha=0.5)
        plt.axis([0,dims[1],dims[0],0])
        plt.xticks([])
        plt.yticks([])

        plt.draw()
        plt.savefig(f'{root}{subfolder}heatmap_{i}.png', bbox_inches='tight', pad_inches=0)

        plt.close(fig)

In [ ]:
from PIL import Image

scenes = [1, 2, 3]
images = [1, 2]
experiences = ['CSI', 'Control', 'FirstYear', 'ThirdYear']
amounts = [10, 25, 50]

pad = "../../data/csv/"
image_pad =  "../../data/Images/CrimeScenes/"

for image in images:
    for scene in scenes:
        
        img = Image.open(image_pad + f'scene_{scene}_{image}.png')
        width, height = img.size
        
        #size of the display
        maxX = width
        maxY = height
        	
        
        #width of the kernel
        gwh = int(maxX/25)
        
        filename = image_pad + f'scene_{scene}_{image}.png' #background image
        fh1 = open(pad + 'fix_evidence.csv', 'r') #open the fixations for trust task
        fix_full = pd.read_csv(fh1, sep=',')
        fix = fix_full[(fix_full['Image'] == image) & (fix_full['Scene'] == scene)]
        
        #set the background image for the image to show (jpg)
        background = np.zeros((maxY,maxX,3),dtype=np.float32)
        #layer=255*np.ones((maxY,maxX),dtype=np.float32)
        #background[:,:,3]=layer
        
        #load the background image
        im = np.array(mpimg.imread(filename))
        dims = im.shape
        print(dims)
        
        #find where to place image in background (center of image)
        startX=int(round((maxY-dims[0])/2))
        startY=int(round((maxX-dims[1])/2))
        
        #put image on background
        background[startX:startX+dims[0], startY:startY+dims[1],:] = im
        
        #create the kernel (Pygaze)
        gaus = gaussian(int(gwh), int(gwh), int(gwh/6))
        strt = int(gwh/2)
        heatmapsize = maxY + 2*strt, maxX + 2*strt

        for exp in experiences:
            heatmap_section(exp, image, scene, 25)

(768, 713, 3)
1693
67
25
1693 0 67
1693 67 134
1693 134 201
1693 201 268
1693 268 335
1693 335 402
1693 402 469
1693 469 536
1693 536 603
1693 603 670
1693 670 737
1693 737 804
1693 804 871
1693 871 938
1693 938 1005
1693 1005 1072
1693 1072 1139
1693 1139 1206
1693 1206 1273
1693 1273 1340
1693 1340 1407
1693 1407 1474
1693 1474 1541
1693 1541 1608
1693 1608 1675
1618
64
25
1618 0 64
1618 64 128
1618 128 192
1618 192 256
1618 256 320
1618 320 384
1618 384 448
1618 448 512
1618 512 576
1618 576 640
1618 640 704
1618 704 768
1618 768 832
1618 832 896
1618 896 960
1618 960 1024
1618 1024 1088
1618 1088 1152
1618 1152 1216
1618 1216 1280
1618 1280 1344
1618 1344 1408
1618 1408 1472
1618 1472 1536
1618 1536 1600
1278
51
25
1278 0 51
1278 51 102
1278 102 153
1278 153 204
1278 204 255
1278 255 306
1278 306 357
1278 357 408
1278 408 459
1278 459 510
1278 510 561
1278 561 612
1278 612 663
1278 663 714
1278 714 765
1278 765 816
1278 816 867
1278 867 918
1278 918 969
1278 969 1020
1278 1020 1071